# Resources

[SELEX Example][selex example]: Motif Discovery from SELEX-Seq data. 

<div class="alert alert-block alert-info"><b>Credit:</b> This notebook is adapted from the SELEX example indicated above. </div>

[SELEX Manual][selex manual]

[Paper introducing SELEX][selex paper]: Cofactor binding evokes latent differences in DNA binding specificity between Hox proteins, Slattery et al., 2011

[selex example]: https://bioconductor.org/packages/release/bioc/vignettes/SELEX/inst/doc/SELEX.pdf
[selex manual]: https://bioconductor.org/packages/release/bioc/manuals/SELEX/man/SELEX.pdf
[selex paper]: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3319069/

# Import libraries

Use either import statement - the one above shows no success statements and no warnings, which are typically not required. If something does not work, out-comment the first line and un-comment the second to see if all required packages and the packages they in turn depend on are imported without any problems.

In [1]:
setwd("/fast/AG_Ohler/jmuino/FlowerDevGroup/Peilin/Claudia/SELEX-seqAnalysis")#write your workinfdirectory hear
suppressMessages(library(SELEX))


<div class="alert alert-block alert-success"> 
    
# How to use this notebook
    
This notebook can perform all the steps required for data analysis of SELEX-seq data with the SELEX package. To use it, follow these steps: 

### 1. Input variables
All file paths, file names and as well as information such as the variable length need to be **given as input variables** in the section indicated below. At the very end of this notebook there is a blue box giving detailed information on how to download and save data sets from the Gene Expression Omnibus, how to interpret the FASTQ files and how to save time by saving the metadata in an annotation file.
    
### 2. Setting up SELEX
The next steps require **no input - just run the cells**. Appropriate settings are set and the data is imported properly such that the SELEX package 'knows' how to deal with and address the data you have provided. 
    
### 3. Analysis
This step requires one input - indicate the round that is to be compared to R0 for the calculation of the information gain and affinities.
    
</div>

<div class="alert alert-block alert-warning"> 

In the following, all sections requiring input are highlighted in yellow.
    
</div>

<div class="alert alert-block alert-warning">

# 1. Input Variables
    
### Protein Name & Round names & Sample Name
Indicate the name of the protein being analyzed, as a string. We will use this string to name samples and output data. Likewise, indicate the name of the sample sequences the protein is analyzed with.

</div>

In [2]:
pname = 'HDG3'
sname = 'RandomSequences'

<div class="alert alert-block alert-warning">

### File paths
    
</div>

In [3]:
r0_filepath = '/fast/AG_Ohler/jmuino/FlowerDevGroup/Peilin/parSelex-seq/ProcessedFASTA/FUL-FUL_R0_FASTERIS.fasta'
#r1_filepath = '/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R1.fastq'
#r2_filepath = '/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R2.fastq'
r3_filepath = '/fast/AG_Ohler/jmuino/FlowerDevGroup/Peilin/parSelex-seq/ProcessedFASTA/HDG3-FLAG_R3_FASTERISAugust.fasta'

<div class="alert alert-block alert-warning">

### Variable length, Left and Right Barcodes
This is the length of the variable region of the nucleic acid snippets in the sample. If sequences are purely random with no defined left or right barcodes, the length of the variable region is equal to the length of the sequence.
    
</div>

In [4]:
variable_length = 20
left_bc = ''
right_bc = ''

# 2.a) Required settings for SELEX

In [5]:
options(java.parameters="-Xmx2500M") # Set maximum memory usage limit

The working directory contains the temporary files. The folder is created if it does not exist, everything in it is automatically kept track of and does not need to be read or addressed by a person. Once calculation is complete, the contents or even the whole folder can safely be deleted. 

Note: Never create a backup of this folder (i.e. with Git), it is huge and upload will take forever. 

In [6]:
workDir = "./cache/" # Specify the temporal working directory

In [7]:
selex.config(workingDir=workDir, maxThreadNumber=4) # Configure Selex

If rJava is propery initialized and installed, the method jvmStatus should show you the current memory usage of the Java Virtual Machine. You can check this if you run into any trouble.

In [8]:
#selex.jvmStatus()

# 2.b) Import your own data

Detailed instructions for downloading data that is available on the GEO are given below, in the last cell in this notebook.

#### Define metadata

This step is required to save metadata such as the filepath, barcodes and length of the variable region. This information is necessary for SELEX to interpret the data in subsequent steps. 

In [9]:
sample_r0 = selex.defineSample(seqName = paste(pname, 'R0', sep='-'), seqFile = r0_filepath, 
                               sampleName = paste('R0', sname, sep='.'), round = 0, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

#sample_r1 = selex.defineSample(seqName = paste(pname, 'R1', sep='-'), seqFile = r1_filepath, 
                               sampleName = paste('R1', sname, sep='.'), round = 1, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

#sample_r2 = selex.defineSample(seqName = paste(pname, 'R2', sep='-'), seqFile = r2_filepath, 
                               sampleName = paste('R2', sname, sep='.'), round = 2, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

Comment out the following cell if there is no third round data set:

In [10]:
sample_r3 = selex.defineSample(seqName = paste(pname, 'R3', sep='-'), seqFile = r3_filepath, 
                               sampleName = paste('R3', sname, sep='.'), round = 3, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

#### Save metadata (i.e. annotation) for later

Print the name to check formatting. (Folders need to be separated by slashes (/) and suffixes need to be preceded by points.)

In [11]:
export_name = paste(paste('', pname, sep=''), 'xml', sep='.')

export_name

[1] "/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1.xml"

In [12]:
selex.saveAnnotation(export_name)

#### Import the samples

In [13]:
r0 = selex.sample(seqName = paste(pname, 'R0', sep='-'), sampleName = paste('R0', sname, sep='.'), round = 0)
#r1 = selex.sample(seqName = paste(pname, 'R1', sep='-'), sampleName = paste('R1', sname, sep='.'), round = 1)
#r2 = selex.sample(seqName = paste(pname, 'R2', sep='-'), sampleName = paste('R2', sname, sep='.'), round = 2)

Comment out the following cell if there is no third round data set:

In [14]:
#r3 = selex.sample(seqName = paste(pname, 'R3', sep='-'), sampleName = paste('R3', sname, sep='.'), round = 3)

#### Split the first sample into a training and a testing part

Default ratio is 0.5 (i.e. half training and half testing data). 

In [15]:
r0.split = selex.split(r0, ratio=c(0.5,0.5))

In [16]:
names(r0.split)

[1] "test"  "train" "info"

In [17]:
r0.split$test

[1] "Java-Object{config.ExperimentReference@71e9ddb4}"

#### View imported data

In [18]:
selex.sampleSummary()

,seqName,sampleName,rounds,leftBarcode,rightBarcode,leftFlank,rightFlank,seqFile
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,SEP3-AGe1C1-R0,R0.RandomSequences,0,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R0.fastq
6,SEP3-AGe1C1-R0.split.1,R0.RandomSequences.split.1,0,,,,,/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.R0.RandomSequences.0_split//0.part
5,SEP3-AGe1C1-R0.split.2,R0.RandomSequences.split.2,0,,,,,/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.R0.RandomSequences.0_split//1.part
4,SEP3-AGe1C1-R1,R1.RandomSequences,1,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R1.fastq
2,SEP3-AGe1C1-R2,R2.RandomSequences,2,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R2.fastq
3,SEP3-AGe1C1-R3,R3.RandomSequences,3,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R3.fastq


# 3. Analysis

<div class="alert alert-block alert-warning">
    
Indicate **which round is to be analyzed**, always in comparison to round zero (R0). Store the handle in the variable "Round2BAnalyzed" (the same name by which SELEX addresses the sample) and store the name as a string (required for saving the output).
    
</div>

In [40]:
Round2BAnalyzed = r3
RoundName = 'R3'

## Markov model

Even though random sequences of DNA are used, the frequency with which they occur is not completely free of bias - i.e. not all random sequences are equally likely to be found.

The SELEX method uses a Markov model to predict the number of occurences of each possible k-mer in round zero (R0).

### 3.1 Count $k_{max}$

Find the maximal nucleotide length $k_{max}$ such that all sequences of that length $k_{max}$ appear at least $n$ times in the data, where $n$ is the threshold value (default: 100). 

The model will be fitted to sequences of length $k_{max}$, all other sequences will be neglected for the purpose of fitting the Markov Model. Therefore, it is important that $n$ be large enough to allow for a statistically sound fit. 

In [20]:
# Find the longest length k such that all possible k-mers are found at least 100 times
kmax.value = selex.kmax(sample=r0.split$test, threshold=100, seqfilter=NULL)

Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 1 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 2 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 3 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 4 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 5 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 6 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 7 ]
[ sample id : SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,  filter:  variableRegionIncludeRegex:null,variableRegionExcludeRegex:null,variableRegionGroupRegex:null ]
[ SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0.kmax = 6 ]


In [21]:
kmax.value

[1] 6

### 3.2 Create and store the Markov model

Next, training and testing data as well as $k_{max}$ is passed to SELEX which creates and saves a Markov model. If the order is not specified, it will test Markov models of increasing order, selecting the optimal order.

The order of the Markov model refers to the number of nucleotides considered when inferring the conditional occurence probabilities of subsequent nucleotides, e.g. P(A|GATTC) for a Markov model of the fifth order.

In [22]:
# Requires samples to be used for training and testing, as well as the k_max value.
mm = selex.mm(sample=r0.split$train, order=NA, crossValidationSample=r0.split$test, Kmax=kmax.value)

Overwriting Kmax = 6
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 1 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 2 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 3 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 4 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 5 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 6 ]
[ markovLength = 5 ]
[ maxR = 0.997453 ]
[ Model = MarkovModelInfo [markovLength=5, markovLengthTotalCount=5914400, markovR2=0.9974534255596722, markovCountsPath=/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.5.dat_A7FE7F4E2E78A43F892C7F3227FFA520, markovObjPath=/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.5.dat_A7FE7F4E2E78A43F892C7F3227FFA520.prob.obj2, sample=config.ExperimentReference@455b6df1, markovModelMethod=DIVISION, crossValidationSample=c

In [23]:
selex.mmSummary()

Sample,Order,MarkovModelType,R,CVSample,CVLength
<chr>,<dbl>,<chr>,<dbl>,<chr>,<int>
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,0,DIVISION,0.9698371,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,1,DIVISION,0.9886711,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,2,DIVISION,0.9947389,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,3,DIVISION,0.9966505,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,4,DIVISION,0.9974534,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,5,DIVISION,0.9971954,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6


### 3.3 Information gain and optimal length

The last line printed is the maximal information gain. 

<div class="alert alert-block alert-danger">
    
If the following line throws **this error**: 
    
>Error in .jcall("RJavaTools", "Ljava/lang/Object;", "invokeMethod", cl, : java.lang.OutOfMemoryError: Java heap space
    
It helps to delete the whole cache (working directory) and then restart the notebook (using the >> Button in the menu). 

</div>

In [24]:
selex.infogain(Round2BAnalyzed, markovModel=mm)

Counting [InfoGain][ K = 5 ]
Counting [InfoGain][ K = 6 ]
Counting [InfoGain][ K = 7 ]
Counting [InfoGain][ K = 8 ]
Counting [InfoGain][ K = 9 ]
Counting [InfoGain][ K = 10 ]
Counting [InfoGain][ K = 11 ]
Counting [InfoGain][ K = 12 ]
Counting [InfoGain][ K = 13 ]
Counting [InfoGain][ K = 14 ]
Counting [InfoGain][ K = 15 ]
Counting [InfoGain][ K = 16 ]


[1] 1.526408

Check which value of k corresponds to the maximal information gain: 

In [25]:
selex.infogainSummary()

Sample,K,InformationGain,MarkovModel,MarkovModelType
<chr>,<int>,<dbl>,<chr>,<chr>
SEP3-AGe1C1-R2.R2.RandomSequences.2,5,0.3477520,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,6,0.5562881,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,7,0.8288981,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,8,1.1434254,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,9,1.4183313,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,10,1.5264085,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,11,1.3981736,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,12,1.0429063,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,13,0.5486344,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION


<div class="alert alert-block alert-warning">

The **optimal length** is the value of k for which the information gain is highest. Save that information in the variable below:
    
</div>

In [26]:
optimalLength = 10

Count how many times k-mers of optimal length appear: 

In [33]:
table = selex.counts(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 

Counting [SEP3-AGe1C1-R2.R2.RandomSequences.2][ K = 10 ]
[ Lowest Count =  1 ]


In [28]:
head(table)

,Kmer,ObservedCount,Probability,ExpectedCount
,<chr>,<int>,<dbl>,<dbl>
1,AAATAAGGAA,11740,5.313668e-06,71.39581
2,AAAAGAGGAA,10422,5.470747e-06,73.50637
3,AAAAAAGGAA,9388,8.423538e-06,113.18082
4,CAAATAAGGA,9142,3.327236e-06,44.70560
5,ATTTAAGGAA,8534,2.286702e-06,30.72471
6,CCAAATAAGG,7988,2.426840e-06,32.60764


### Calculating affinity and error

Affinities are calculated for specific rounds, so the sample needs to be specified appropriately (i.e. enter r2 for round 2 or r3 for round 3, etc.)

In [34]:
aff = selex.affinities(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 

Counting [SEP3-AGe1C1-R2.R2.RandomSequences.2][ K = 10 ]
[ Lowest Count =  1 ]


In [35]:
head(aff)

,Kmer,ObservedCount,Probability,ExpectedCount,Affinity,SE
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,AAATAAGGAA,11740,5.313668e-06,71.39581,0.4965242,0.006480692
2,AAAAGAGGAA,10422,5.470747e-06,73.50637,0.4610583,0.006386976
3,AAAAAAGGAA,9388,8.423538e-06,113.18082,0.3526492,0.005147204
4,CAAATAAGGA,9142,3.327236e-06,44.70560,0.5537101,0.008189867
5,ATTTAAGGAA,8534,2.286702e-06,30.72471,0.6453201,0.009879020
6,CCAAATAAGG,7988,2.426840e-06,32.60764,0.6060411,0.009589547


### Save and export the affinities table

In [45]:
export_table_name = paste(paste('', pname, sep=''), RoundName, 'affinities.csv', sep='_')

export_table_name

[1] "data/SEP3-AGe1C1_R2_affinities.csv"

In [46]:
write.csv(aff, export_table_name)

<div class="alert alert-block alert-info">

# Instructions on how to download and import your own data set

### Download

#### Accessing data via the gene expression omnibus (GEO)

Data sets can be found on GEO. (Learn more about GEO: https://www.ncbi.nlm.nih.gov/geo/info/faq.html#what)

Data sets have a **GEO access code** leading to a page that lists information such as the specific extraction protocol and who to contact if you have questions about the data. One code for a data set of ours is **GSM2522798**, you can use this for the following steps if you want to familiarise yourself with them, or you can use any other access code for a dataset you wish to check. 

1. Go to the GEO start page: https://www.ncbi.nlm.nih.gov/geo/ and **enter the access code** in the search bar on the right. 

2. On the page this leads you to (example: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2522798), scroll all the way down and click on **the link next to "SRA"**, which takes you to a new page.

3. Scroll down again, to the **table at the bottom with a cell called "Run"**. Click on the link given in the cell, it takes you to the *SRA page* for the data set. (View it here: https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR5314587 , SRA stands for *Sequence Read Archive*.) 

4. There are four tabs on the left, called *Metadata, Analysis, Reads and Data access*. Click on the **tab called Reads**, then click on "Filtered Download". 

5. For analysis with SELEX, select the **FASTQ format** and proceed to download the data set (bottom right). 

6. The data set is compressed - you need to unpack it. While you are at it, it is a good idea to change the name to something more descriptive than 'sra_data.fastq', which is the default. Here, I named it **SEP3-AGe1C1-R0.fastq**, which is the name of the sequence (GSM2522798 is the iD, both the name and the iD can be found on the SRA and GEO pages). 

#### What should this file look like? 

If you open such a data set with a text editor, you will see lines upon lines looking like this: 

@SRR5314587.1.1 HWUSI-EAS517:93:FC650FPAAXX:5:1:4588:1064 length=20

TAGTACAATACGTGACGACA

+SRR5314587.1.1 HWUSI-EAS517:93:FC650FPAAXX:5:1:4588:1064 length=20

DDDBD7@@A@*B?BB?@B>C

The first and third line contains the sequence iD and description. The *second line contains the sequence* and the *fourth line contains the quality of the sequence*. For more information, see: https://en.wikipedia.org/wiki/FASTQ_format

#### Repeat the above steps to get the files for all SELEX rounds (R0 through to R2 or R3). Save all filepaths.

For the above example, the other rounds have GEO iDs GSM2522799, GSM2522800, GSM2522801.

### Metadata

Before a **new data set** can be analyzed, it is necessary to pass some information about the data set to SELEX, so that the information contained in the data set can be correctly interpreted. This is done with the **function selex.defineSample**. Required parameters (they need to be given in the correct order): 

- seqName
- seqFile
- sampleName
- round
- varLength
- leftBarcode
- rightBarcode

Detailed documentation can be found in the SELEX manual on page 10. 

In this case, for our sample (GSM2522798), the information is: 

- SEP3-AGe1C1-R0 (the name of the sample, GSM2522798 is the iD, both infos found in SRA or on GEO)
- file path (specified above)
- R0.RandomSequences
- 0
- 20 nt
- no barcode - specify an empty string ('')
- no barcode - specify an empty string ('')

(For subsequent rounds, adjust names and values accordingly.)

Since we cannot distinguish our training & metadata by barcodes, we split the file into two parts and import them separately. 

### Don´t want to do it again?

There is also the option of creating an **annotation file** (XML). Instead of passing information about the data set each time, it is then sufficient to load the annotation file, using the function **selex.loadAnnotation**. 

To create the XML file, simply run **selex.saveAnnotation(filePath)**. It will save all the metadata passed to SELEX in the current session in an XML file. 
In this case, the file will contain the metadata for just one sequence, so the sequence name is an appropriate name for the xml file. 
(In this notebook, this step is performed in section 2.b).)
    
</div>